In [1]:
import tensorflow as tf
x=tf.Variable(3, name='x')
y=tf.Variable(4, name='y')
f = x*x*y + y +2

In [2]:
import tensorflow as tf

x=tf.Variable(3, name='x')
y=tf.Variable(4, name='y')

In [3]:
#with tf.Session() as sess:
 #   x.initializer.run()
  #  y.initializer.run()
   # result =f.eval()
    
#or everything at the sametime

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() 
    result = f.eval()

In [4]:
#if you want to use InteracticeSession it is  automatically sets itself as the default session with block but you do need to close the session manually 


sess= tf.InteractiveSession()
init.run()
result =f.eval()

print(result)



42


In [5]:
sess.close()


# Managing Graphs



In [6]:
x1= tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [7]:
graph = tf.Graph()
with graph.as_default():
    x2=tf.Variable(2)
    
x2.graph is graph

True

In [8]:
x2.graph is tf.get_default_graph()


False

# Lifecycle of a Node Value


In [9]:
w=tf.constant(3)
x=w+2
y=x+5
z=y+3


In [10]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
13


In [11]:
with tf.Session() as sess:
    y_val, x_val = sess.run([y,z])
    print(y_val)
    print(x_val)

10
13


# Linear Regression with 

In [12]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing=fetch_california_housing() # data

m,n= housing.data.shape

housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

X=tf.constant(housing_data_plus_bias, dtype =tf.float32, name="X")
y= tf.constant(housing.target.reshape(-1,1), dtype=tf.float32, name="y")
XT=tf.transpose(X)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT,X)), XT), y)


with tf.Session() as sess:
    theta_value = theta.eval()


# Implementing Gradient Descent

# 1_ Manually Computing the Gradients

In [13]:
from sklearn.preprocessing import StandardScaler  

n_epochs = 1000
learning_rate = 0.01
#not from the book
scaler = StandardScaler()  
scaled_housing_data = scaler.fit_transform(housing.data)  
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]  
#--- 

X= tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y =tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1,1], -1.0,1.0), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")

error = y_pred - y

mse = tf.reduce_mean(tf.square(error), name= "mse")

gradients = 2/m * tf.matmul(tf.transpose(X), error)

training_op = tf.assign(theta, theta-learning_rate*gradients)

init= tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(n_epochs):
        if epoch % 100 ==0: 
            print("Epoch", epoch, "MSE= ", mse.eval())
            
        sess.run(training_op)
    best_theta =theta.eval()


Epoch 0 MSE=  6.24192
Epoch 100 MSE=  0.698669
Epoch 200 MSE=  0.591705
Epoch 300 MSE=  0.576163
Epoch 400 MSE=  0.565351
Epoch 500 MSE=  0.556964
Epoch 600 MSE=  0.550407
Epoch 700 MSE=  0.545254
Epoch 800 MSE=  0.541184
Epoch 900 MSE=  0.537954


# 2_Using autodiff

In [14]:


gradients = tf.gradients(mse, [theta])[0]



# 3_ Using an optimizer

In [15]:

optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)

training_op = optimizer.minimize(mse)

optimizer = tf.train.MomentumOptimizer(learning_rate = learning_rate, momentum = 0.9)

# Feeding Data to the Training Algorithm


In [16]:
A = tf.placeholder(tf.float32, shape=(None, 3))

B= A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict = {A : [[1,2,3]]})
    B_val_2 = B.eval(feed_dict = {A : [[4,5,6],[7,8,9]]})
    
print(B_val_1)

print(B_val_2)

[[ 6.  7.  8.]]
[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [17]:
X=tf.placeholder(tf.float32, shape=(None, n+1), name = "X")
y= tf.placeholder(tf.float32, shape = (None, 1), name="y")

batch_size = 100


n_batches = int(np.ceil(m / batch_size))


"""
>>> a = np.array([-1.7, -1.5, -0.2, 0.2, 1.5, 1.7, 2.0])
>>> np.ceil(a)
array([-1., -1., -0.,  1.,  2.,  2.,  2.])
"""
print(n_batches)

207


In [18]:
"""
def fetch_batch(epoch, batch_index, batch_size):
    [...] #load the data from disk
    
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run (training_op, feed_dict = {X : X_batch, y: y_batch})
        best_theta = theta.eval()
        
        """

'\ndef fetch_batch(epoch, batch_index, batch_size):\n    [...] #load the data from disk\n    \n    return X_batch, y_batch\n\nwith tf.Session() as sess:\n    sess.run(init)\n    \n    for epoch in range(n_epochs):\n        for batch_index in range(n_batches):\n            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)\n            sess.run (training_op, feed_dict = {X : X_batch, y: y_batch})\n        best_theta = theta.eval()\n        \n        '

# Saving and Restoring Models



In [20]:
theta = tf.Variable(tf.random_uniform([n+1,1], -1.0, 1.0), name= "theta")

init = tf.global_variables_initializer()

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0: #checkpoint every 100 epochs
            save_path = saver.save(sess, "/home/wonseok/Desktop/l/coding/9.Hands-On_machine_learning//PART2//save//my_model.ckpt")
        sess.run(training_op)
        
    best_theta = theta.eval()
    save_path = saver.save(sess, "/home/wonseok/Desktop/l/coding/9.Hands-On_machine_learning//PART2//save//my_model_final.ckpt")


In [21]:
#restoring a model 

with tf.Session() as sess:
    saver.restore(sess, "..//coding//9.Hands-On_machine_learning//PART2//save//my_model_final.ckpt")


    #will save or restore only the theta variable under the name weight
saver=tf.train.Saver({"weights" : theta})

InvalidArgumentError: Unsuccessful TensorSliceReader constructor: Failed to get matching files on ..//coding//9.Hands-On_machine_learning//PART2//save//my_model_final.ckpt: Not found: ..//coding//9.Hands-On_machine_learning//PART2//save/
	 [[Node: save_1/RestoreV2_7 = RestoreV2[dtypes=[DT_INT32], _device="/job:localhost/replica:0/task:0/cpu:0"](_recv_save_1/Const_0, save_1/RestoreV2_7/tensor_names, save_1/RestoreV2_7/shape_and_slices)]]

Caused by op 'save_1/RestoreV2_7', defined at:
  File "/home/wonseok/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/wonseok/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/wonseok/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/wonseok/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/w

# Visulaizing the Graph and Training Curves Using TensorBoard

In [22]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")

root_logdir = "tf_logs"

logdir = "{}/run-{}/".format(root_logdir, now)

mse_summary = tf.summary.scalar('MSE', mse)

file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

for batch_index in range(n_batches):
    X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
    if batch_index % 10 == 0:
        summary_str = mse_summary.eval(feed_dict = {X : X_batch, y: y_batch})
        step = epoch * n_batches + batch_index
        file_writer.add_summary(summary_str, step)
    sess.run(training_op, feed_dict = {X:X_batch, y: y_batch})
    

NameError: name 'fetch_batch' is not defined

In [ ]:
#clost the FileWriter at the end of the program

file_writer.close()

In [ ]:
def conv_basic(_input, _w, _b, _keepratio, _use_gray):
    # INPUT
    if _use_gray:
        _input_r = tf.reshape(_input, shape=[-1, imgsize[0], imgsize[1], 1])
    else:
        _input_r = tf.reshape(_input, shape=[-1, imgsize[0], imgsize[1], 3])
    # CONVOLUTION LAYER 1
    _conv1 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_input_r
        , _w['wc1'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc1']))
    _pool1 = tf.nn.max_pool(_conv1, ksize=[1, 2, 2, 1]
        , strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr1 = tf.nn.dropout(_pool1, _keepratio)
    # CONVOLUTION LAYER 2
    _conv2 = tf.nn.relu(tf.nn.bias_add(tf.nn.conv2d(_pool_dr1
        , _w['wc2'], strides=[1, 1, 1, 1], padding='SAME'), _b['bc2']))
    _pool2 = tf.nn.max_pool(_conv2, ksize=[1, 2, 2, 1]
        , strides=[1, 2, 2, 1], padding='SAME')
    _pool_dr2 = tf.nn.dropout(_pool2, _keepratio)
    # VECTORIZE
    _dense1 = tf.reshape(_pool_dr2
                         , [-1, _w['wd1'].get_shape().as_list()[0]])
    # FULLY CONNECTED LAYER 1
    _fc1 = tf.nn.relu(tf.add(tf.matmul(_dense1, _w['wd1']), _b['bd1']))
    _fc_dr1 = tf.nn.dropout(_fc1, _keepratio)
    # FULLY CONNECTED LAYER 2
    _out = tf.add(tf.matmul(_fc_dr1, _w['wd2']), _b['bd2'])
    # RETURN
    out = {
        'input_r': _input_r, 'conv1': _conv1, 'pool1': _pool1
        , 'pool1_dr1': _pool_dr1, 'conv2': _conv2, 'pool2': _pool2
        , 'pool_dr2': _pool_dr2, 'dense1': _dense1, 'fc1': _fc1
        , 'fc_dr1': _fc_dr1, 'out': _out
    }
    return out
print ("NETWORK READY")

In [ ]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_output])
keepratio = tf.placeholder(tf.float32)

# Functions! 
_pred = conv_basic(x, weights, biases, keepratio, use_gray)['out']
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(_pred, y))
WEIGHT_DECAY_FACTOR = 0.0001
l2_loss = tf.add_n([tf.nn.l2_loss(v) 
            for v in tf.trainable_variables()])
cost = cost + WEIGHT_DECAY_FACTOR*l2_loss
optm = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
_corr = tf.equal(tf.argmax(_pred,1), tf.argmax(y,1)) # Count corrects
accr = tf.reduce_mean(tf.cast(_corr, tf.float32)) # Accuracy
init = tf.initialize_all_variables()
print ("FUNCTIONS READY")

In [ ]:

# Parameters
training_epochs = 400
batch_size      = 100
display_step    = 40

# Launch the graph
sess = tf.Session()
sess.run(init)

# Training cycle
for epoch in range(training_epochs):
    avg_cost = 0.
    num_batch = int(ntrain/batch_size)+1
    # Loop over all batches
    for i in range(num_batch): 
        randidx = np.random.randint(ntrain, size=batch_size)
        batch_xs = trainimg[randidx, :]
        batch_ys = trainlabel[randidx, :]                
        # Fit training using batch data
        sess.run(optm, feed_dict={x: batch_xs, y: batch_ys
                                  , keepratio:0.7})
        # Compute average loss
        avg_cost += sess.run(cost, feed_dict={x: batch_xs, y: batch_ys
                                , keepratio:1.})/num_batch

    # Display logs per epoch step
    if epoch % display_step == 0 or epoch == training_epochs-1:
        print ("Epoch: %03d/%03d cost: %.9f" % 
               (epoch, training_epochs, avg_cost))
        train_acc = sess.run(accr, feed_dict={x: batch_xs
                                , y: batch_ys, keepratio:1.})
        print (" Training accuracy: %.3f" % (train_acc))
        test_acc = sess.run(accr, feed_dict={x: testimg
                                , y: testlabel, keepratio:1.})
        print (" Test accuracy: %.3f" % (test_acc))
print ("Optimization Finished!")

In [ ]:
sess.close()
print ("Session closed.")

In [ ]:
#http://tensorflowstepbystep.tistory.com/4

#tensorboard --logdir=path/to/log-directory